In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
from tests.Model import Model
from tests.rigid_utils import Rigid

In [3]:
# of_basic_features = torch.load('kilian/basic_features.pt', map_location='cpu')
# 
# extra_msa_feat = torch.load('kilian/extra_msa_feat.pt', map_location='cpu')
# msa_feat = of_basic_features['msa_feat']
# residue_index = of_basic_features['residue_index']
# target_feat = of_basic_features['target_feat']

c_z = 128
c_m  = 256
tf_dim = 22
openfold_param_path = ('tests/openfold_params/finetuning_ptm_2.pt')
openfold_weights = torch.load(openfold_param_path, map_location='cpu')

# batch = {'extra_msa_feat': extra_msa_feat, 'msa_feat': msa_feat, 'residue_index': residue_index, 'target_feat': target_feat}
batch = torch.load('tests/test_outputs/my_batch.pt', map_location='cuda')

In [4]:
def remap_weight_names(openfold_weights):
    name_map = {
        'core.msa_transition': 'msa_transition',
        'core.outer_product_mean': 'outer_product_mean',
        'msa_att_col._msa_att': 'msa_att_col',
        'transition.layers.0.linear_1': 'transition.linear_1',
        'transition.layers.0.linear_2': 'transition.linear_2',
        'transition.layers.0.linear_3': 'transition.linear_3',
    }
    for key in list(openfold_weights.keys()):
        new_key = None
        for sub in name_map.keys():
            if sub in key:
                new_key = key.replace(sub, name_map[sub])
        
        if 'structure_module.ipa.linear_kv.'in key:
            value = openfold_weights[key]
            if 'weight' in key:
                value = value.T
            n_head=12
            c=16
            value = value.view(value.shape[:-1]+(n_head,2*c))
            k, v = value.split(c, dim=-1)
            if 'weight' in key:
                openfold_weights['structure_module.ipa.linear_k.weight'] = k.flatten(start_dim=-2).T
                openfold_weights['structure_module.ipa.linear_v.weight'] = v.flatten(start_dim=-2).T
            else:
                openfold_weights['structure_module.ipa.linear_k.bias'] = k.flatten(start_dim=-2)
                openfold_weights['structure_module.ipa.linear_v.bias'] = v.flatten(start_dim=-2)

            openfold_weights.pop(key)

        if 'structure_module.ipa.linear_kv_points.' in key:
            value=openfold_weights[key]
            if 'weight' in key:
                value=value.T
            n_head=12
            query_points=4
            value_points=8
            value = value.view(value.shape[:-1] + (3*n_head, query_points+value_points))
            k, v = value.split([query_points, value_points], dim=-1)
            if 'weight' in key:
                openfold_weights['structure_module.ipa.linear_k_points.weight'] = k.flatten(start_dim=-2).T
                openfold_weights['structure_module.ipa.linear_v_points.weight'] = v.flatten(start_dim=-2).T
            else:
                openfold_weights['structure_module.ipa.linear_k_points.bias'] = k.flatten(start_dim=-2)
                openfold_weights['structure_module.ipa.linear_v_points.bias'] = v.flatten(start_dim=-2)
            openfold_weights.pop(key)

        if new_key is not None:
            openfold_weights[new_key] = openfold_weights.pop(key)

remap_weight_names(openfold_weights)
            

In [5]:
for key, tensor in batch.items():
    batch[key] = tensor.to('cuda')

In [6]:
model = Model()
res = model.load_state_dict(openfold_weights, strict=False)
print(type(res))
missing_struct = [a for a in res.missing_keys if 'structure' in a]
incompatible = [a for a in res.unexpected_keys if 'template' not in a]

for key in incompatible:
    if 'template' not in key and 'aux' not in key:
        print(key)
        print(torch.abs(openfold_weights[key]).mean())


model.to('cuda')
model.eval()
print(next(model.parameters()).device)

<class 'torch.nn.modules.module._IncompatibleKeys'>
cuda:0


In [8]:
s = torch.load('tests/test_outputs/s_evo.pt', map_location='cuda')
z = torch.load('tests/test_outputs/z_evo.pt', map_location='cuda')
F = batch['target_feat'].argmax(dim=-1) - 1
F = F.to('cuda')
with torch.no_grad():
    outputs = model(batch)

Starting iteration 0...
Starting iteration 1...
Starting iteration 2...
Starting iteration 3...


In [10]:
my_angles = outputs['angles'][-1,...]
my_frames = outputs['frames'][-1,...]
my_positions = outputs['final_positions'][-1,...]
my_mask = outputs['position_mask'][-1,...]
print(my_positions.shape)

of_angles = torch.load('tests/test_outputs/final_angles.pt')
of_frames = torch.load('tests/test_outputs/final_frames.pt')
of_positions = torch.load('tests/test_outputs/final_positions.pt')
of_mask = torch.load('tests/test_outputs/final_atom_mask.pt')

torch.Size([59, 37, 3])


In [11]:
rigid = Rigid.from_tensor_7(of_frames)
of_rot_mats = rigid.get_rots().get_rot_mats()
of_trans = rigid.get_trans()
print(of_rot_mats.shape, of_trans.shape)


torch.Size([8, 59, 3, 3]) torch.Size([8, 59, 3])


In [12]:
print(torch.abs(of_rot_mats-my_frames[...,:3,:3]).mean())
print(torch.abs(of_trans-my_frames[...,:3,3]*10).mean())
print(torch.abs(of_angles-my_angles/torch.linalg.vector_norm(my_angles, dim=-1, keepdim=True)).mean())

print(my_positions.shape, my_mask.shape)
my_positions = my_positions * my_mask.unsqueeze(-1)

print((my_positions-of_positions).abs().mean() / of_positions.abs().mean())


tensor(1.3236e-07, device='cuda:0')
tensor(9.4366e-07, device='cuda:0')
tensor(7.0798e-08, device='cuda:0')
torch.Size([59, 37, 3]) torch.Size([59, 37])
tensor(1.9242e-07, device='cuda:0')


In [ ]:
import modelcif
import modelcif.model
import modelcif.dumper
import io
from kilian.structure_module.residue_constants import atom_types

def to_modelcif(atom_positions, atom_mask, sequence):
    atom_positions = atom_positions.to_numpy()
    atom_mask = atom_mask.to_numpy()
    n = atom_positions.shape[0]
    system = modelcif.System(title='AlphaFold prediction')
    entity = modelcif.Entity(sequence, description='Model subunit')
    asym_unit = modelcif.AsymUnit(entity, details='Model subunit A', id='A')
    modeled_assembly = modelcif.Assembly([asym_unit], name='Modeled assembly')
    class _MyModel(modelcif.model.AbInitioModel):
        def get_atoms(self):
            for i in range(n):
                for atom_name, pos, mask in zip(atom_types, atom_positions[i], atom_mask[i]):
                    if not mask:
                        continue
                    element = atom_name[0]
                    yield modelcif.model.Atom(
                        asym_unit=asym_unit,
                        type_symbol=element,
                        seq_id=i+1,
                        atom_id=atom_name,
                        x=pos[0], y=pos[1], z=pos[2],
                        het=False,
                        occupancy=1.00
                    )

    model = _MyModel(assembly=modeled_assembly, name='Model')
    model_group = modelcif.model.ModelGroup([model], name='All models')
    system.model_groups.append(model_group)
    fh = io.StringIO()
    modelcif.dumper.write(fh, [system])
    return fh.getvalue()
            

In [ ]:
sequence = 'PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK'
with open('./kilian/prediction.cif', 'w') as f:
    f.write(to_modelcif(my_positions.to('cpu'), my_mask.to('cpu'), sequence))